In [2]:
%%time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from dataclasses import dataclass, field
from typing import Optional
import contextlib

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)


from trl import SFTTrainer

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype="bfloat16",
#     bnb_4bit_use_double_quant=False,
# )

# device_map = {"": 0}
base_model = "bigcode/starcoderbase-1b"
model = "smangrul/full-finetune-starcoderbase-3b-deepspeed-colab"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model, quantization_config=None, 
    device_map=None, 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16,
)

if not hasattr(model, "hf_device_map"):
    model.cuda()

CPU times: user 21.7 s, sys: 5.67 s, total: 27.4 s
Wall time: 1min 19s


In [8]:
def get_code_completion(prefix, suffix):
    text = prompt = f"""<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>"""
    model.eval()
    outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids.cuda(), 
                             max_new_tokens=512,
                             temperature=0.3,
                             top_k=50,
                             top_p=0.95,
                             do_sample=True,
                             repetition_penalty=1.2,
                             #stopping_criteria=stopping_criteria
                            )
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

In [11]:
prefix = """
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
"""

suffix = """
two_sum([3,4,5], 7)
"""
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
<fim_suffix>
two_sum([3,4,5], 7)
<fim_middle>    """
    Two Sum Problem

    Args:
        arr (list[int]):
            List containing integers.
        target_sum (int):
            The desired sum for which to find the two smallest elements in an array.

    Returns:
        Tuple[List[int]]: A tuple with the indices of the found values.
    """
    # Initialize our variables
    low = 0
    high = len(arr) - 1

    while low < high:
        mid = (low + high) // 2

        if arr[mid] == target_sum:
            return [mid, mid + 1]
        elif arr[mid] > target_sum:
            high = mid - 1
        else:
            low = mid + 1

    return [low, high + 1]<|endoftext|>


In [13]:
prefix = """from peft import LoraConfig, TaskType

peft_config = LoraConfig("""

suffix = ")"
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from peft import LoraConfig, TaskType

peft_config = LoraConfig(<fim_suffix>)<fim_middle>
    r=8, lora_alpha=32, target_modules=["q", "v"], lora_dropout=0.15, bias="none"
<|endoftext|>


In [16]:
prefix = """from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, training_dataloader, scheduler = """

suffix = """"""
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, training_dataloader, scheduler = <fim_suffix><fim_middle>accelerator.prepare(
    model, optimizer, training_dataloader, scheduler
)
<|endoftext|>
